In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width: 98% !important }</style>'))

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from utilities import *

In [3]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-2], '-')
    return [tf.strings.to_number(parts[-2]), tf.strings.to_number(parts[-1])]

In [4]:
def decode_img(img):
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [5]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [6]:
train_ds = tf.data.Dataset.list_files('../Data/train_images/*/*.png')

In [7]:
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [8]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(32)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [9]:
train_ds = prepare_for_training(train_ds)

In [10]:
test_ds = tf.data.Dataset.list_files('../Data/test_images/*/*.png')

In [11]:
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [12]:
def prepare_for_testing(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(32)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [13]:
test_ds = prepare_for_testing(test_ds)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(17, 34, 1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(2))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 15, 32, 32)        320       
_________________________________________________________________
flatten (Flatten)            (None, 15360)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                983104    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 983,554
Trainable params: 983,554
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

history = model.fit(train_ds, epochs=200, steps_per_epoch=1000, validation_data=test_ds, validation_steps=250)

Train for 1000 steps, validate for 250 steps
Epoch 1/200
1000/1000 [==============================] - 9s 9ms/step - loss: 6.5167 - mae: 1.6550 - mse: 6.5167 - val_loss: 4.0693 - val_mae: 1.5084 - val_mse: 4.0693
Epoch 2/200
1000/1000 [==============================] - 7s 7ms/step - loss: 2.1691 - mae: 1.1457 - mse: 2.1691 - val_loss: 4.1255 - val_mae: 1.5598 - val_mse: 4.1255
Epoch 3/200
1000/1000 [==============================] - 7s 7ms/step - loss: 1.9469 - mae: 1.0775 - mse: 1.9469 - val_loss: 4.3143 - val_mae: 1.6025 - val_mse: 4.3143
Epoch 4/200
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7627 - mae: 1.0183 - mse: 1.7627 - val_loss: 3.3610 - val_mae: 1.4075 - val_mse: 3.3610
Epoch 5/200
1000/1000 [==============================] - 7s 7ms/step - loss: 1.4712 - mae: 0.9175 - mse: 1.4712 - val_loss: 3.5533 - val_mae: 1.5075 - val_mse: 3.5533
Epoch 6/200
1000/1000 [==============================] - 7s 7ms/step - loss: 1.1740 - mae: 0.8110 - mse: 1.1740 - val_lo

1000/1000 [==============================] - 6s 6ms/step - loss: 0.1606 - mae: 0.2680 - mse: 0.1606 - val_loss: 2.5263 - val_mae: 1.1855 - val_mse: 2.5263
Epoch 50/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1627 - mae: 0.2619 - mse: 0.1627 - val_loss: 2.5033 - val_mae: 1.1814 - val_mse: 2.5033
Epoch 51/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1588 - mae: 0.2600 - mse: 0.1588 - val_loss: 2.5521 - val_mae: 1.1928 - val_mse: 2.5521
Epoch 52/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1534 - mae: 0.2582 - mse: 0.1534 - val_loss: 2.5981 - val_mae: 1.2022 - val_mse: 2.5981
Epoch 53/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1570 - mae: 0.2563 - mse: 0.1570 - val_loss: 2.4637 - val_mae: 1.1688 - val_mse: 2.4637
Epoch 54/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1518 - mae: 0.2553 - mse: 0.1518 - val_loss: 2.5372 - val_mae: 1.1943 - val_mse: 2.5372
Epoch

Epoch 98/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1031 - mae: 0.2031 - mse: 0.1031 - val_loss: 2.7362 - val_mae: 1.2288 - val_mse: 2.7362
Epoch 99/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0976 - mae: 0.2020 - mse: 0.0976 - val_loss: 2.9399 - val_mae: 1.2788 - val_mse: 2.9399
Epoch 100/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0938 - mae: 0.1999 - mse: 0.0938 - val_loss: 2.8039 - val_mae: 1.2524 - val_mse: 2.8039
Epoch 101/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1003 - mae: 0.1992 - mse: 0.1003 - val_loss: 2.6682 - val_mae: 1.2178 - val_mse: 2.6682
Epoch 102/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0932 - mae: 0.1999 - mse: 0.0932 - val_loss: 2.7953 - val_mae: 1.2403 - val_mse: 2.7953
Epoch 103/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0957 - mae: 0.1995 - mse: 0.0957 - val_loss: 2.7249 - val_mae: 1.2323 - val_

1000/1000 [==============================] - 6s 6ms/step - loss: 0.0832 - mae: 0.1800 - mse: 0.0832 - val_loss: 2.7552 - val_mae: 1.2423 - val_mse: 2.7552
Epoch 147/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0775 - mae: 0.1792 - mse: 0.0775 - val_loss: 2.8796 - val_mae: 1.2688 - val_mse: 2.8796
Epoch 148/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0740 - mae: 0.1767 - mse: 0.0740 - val_loss: 2.8576 - val_mae: 1.2633 - val_mse: 2.8576
Epoch 149/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0806 - mae: 0.1773 - mse: 0.0806 - val_loss: 2.9855 - val_mae: 1.3005 - val_mse: 2.9855
Epoch 150/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0688 - mae: 0.1726 - mse: 0.0688 - val_loss: 2.7919 - val_mae: 1.2381 - val_mse: 2.7919
Epoch 151/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0796 - mae: 0.1785 - mse: 0.0796 - val_loss: 2.7236 - val_mae: 1.2300 - val_mse: 2.7236


1000/1000 [==============================] - 6s 6ms/step - loss: 0.0701 - mae: 0.1657 - mse: 0.0701 - val_loss: 2.8404 - val_mae: 1.2537 - val_mse: 2.8404
Epoch 195/200
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0580 - mae: 0.1575 - mse: 0.0580 - val_loss: 2.9736 - val_mae: 1.2997 - val_mse: 2.9736
Epoch 196/200
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0676 - mae: 0.1625 - mse: 0.0676 - val_loss: 2.8142 - val_mae: 1.2485 - val_mse: 2.8142
Epoch 197/200
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0672 - mae: 0.1640 - mse: 0.0672 - val_loss: 2.8450 - val_mae: 1.2625 - val_mse: 2.8450
Epoch 198/200
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0603 - mae: 0.1602 - mse: 0.0603 - val_loss: 2.9268 - val_mae: 1.2908 - val_mse: 2.9268
Epoch 199/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0683 - mae: 0.1638 - mse: 0.0683 - val_loss: 2.9560 - val_mae: 1.2911 - val_mse: 2.9560
